## Реализовать алгоритмы поиска подстрок:

### а) Алгоритм Рабина-Карпа.
### б) Алгоритм на основе конечного автомата.

Для первого алгоритма характерно следующее:

1. Вычисляется хэш подстроки и хэш каждого возможного сдвига подстроки в строке.
2. Если хэши совпадают, проводится дополнительная проверка для исключения ложных срабатываний.

In [1]:
# Функция hash_str(s) - это вспомогательная функция, вычисляющая хэш строки s. 
# В данной реализации хэш строки рассчитывается по символам: каждый символ преобразуется в его ASCII-код 
# и добавляется к предыдущему хэшу с умножением на 256

#  В основной функции rabin_karp(text, pattern):
#    - Вычисляются длины входного текста n и шаблона m, а также хэш шаблона pattern_hash и хэш начальной подстроки 
#       текста text[:m].
#    - Проходим по тексту с шагом от 0 до n - m + 1. На каждом шаге сравниваем хэши текущей подстроки текста и шаблона:
#      - Если хэши совпадают и сами подстроки совпадают, возвращаем индекс начала вхождения шаблона в текст.
#      - Если подстрока не совпадает, пересчитываем хэш для следующей подстроки, вычитая ASCII-код символа, 
#        который уходит из окна, и добавляя новый символ.
#    - Если шаблон не найден, возвращается -1.


def rabin_karp(text, pattern):
    # Функция для вычисления хэша строки
    def hash_str(s):
        h = 0
        for char in s:
            h = ord(char) + h * 256
        return h
    
    n = len(text)
    m = len(pattern)
    pattern_hash = hash_str(pattern)
    text_hash = hash_str(text[:m])
    
    # Проход по строке и сравнивание хэшей
    for i in range(n - m + 1):
        if text_hash == pattern_hash and text[i:i+m] == pattern:
            return i
        if i < n - m:
            text_hash = ord(text[i+m]) + (text_hash - ord(text[i]) * 256) * 256
    
    return -1

Второй алгоритм работает путём предварительной обработки подстроки и создания автомата для сопоставления символов подстроки с текстом.

In [2]:
# Функция tr_table(pattern) строит таблицу переходов для конечного автомата, основываясь на шаблоне pattern. 
# В таблице указывается, куда переходить из каждого состояния автомата при определенном символе входной строки.
#    - Создается таблица tr, содержащая словари для каждого состояния и каждого возможного символа.
#    - Инициализируется начальное состояние 0 и переменная prefix для отслеживания префикса.
#    - Для каждого состояния автомата проверяется, какое состояние достигается при текущем символе:
#      - Если символ совпадает с символом шаблона, переходим в следующее состояние.
#      - Иначе переходим в состояние, соответствующее наибольшему совпадающему префиксу шаблона.
#    - Возвращается построенная таблица tr.

#  В главной функции automaton(text, pattern) осуществляется поиск шаблона pattern в тексте text 
# с использованием построенного конечного автомата.
#    - Задаются длины текста n и шаблона m.
#    - Строится таблица переходов tr на основе шаблона.
#    - Устанавливается начальное состояние автомата на 0.
#    - Проходим по каждому символу входного текста:
#      - Обновляем текущее состояние автомата путем считывания символа.
#      - Если мы достигли конечного состояния m, возвращаем позицию начала вхождения шаблона в текст (индекс i - m + 1).
#    - Если шаблон не найден, функция возвращает -1.


def automaton(text, pattern):
    def tr_table(pattern):
        tr = [{char: 0 for char in set(pattern)} for _ in range(len(pattern) + 1)]
        tr[0][pattern[0]] = 1
        prefix = 0
        
        for state in range(1, len(pattern) + 1):
            for char in tr[state].keys():
                if state < len(pattern) and char == pattern[state]:
                    tr[state][char] = state + 1
                else:
                    tr[state][char] = tr[prefix][char]
                    
            prefix = tr[prefix][pattern[state - 1]]
        
        return tr
    
    n = len(text)
    m = len(pattern)
    tr = tr_table(pattern)
    state = 0
    
    for i in range(n):
        state = tr[state].get(text[i], 0)
        if state == m:
            return i - m + 1
    
    return -1

Проверка с использованием unittest

In [3]:
import unittest

class TestAlgorithms(unittest.TestCase):
    
    def test_rabin_karp(self):
        self.assertEqual(rabin_karp("hello", "ll"), 2)
        self.assertEqual(rabin_karp("abcdef", "cde"), 2)
        self.assertEqual(rabin_karp("abcde", "fg"), -1)
    
    def test_automaton(self):
        self.assertEqual(automaton("hello", "ll"), 2)
        self.assertEqual(automaton("abcdef", "cde"), 2)
        self.assertEqual(automaton("abcde", "fg"), -1)

if __name__ == '__main__':
    unittest.main()


E
ERROR: C:\Users\1\AppData\Roaming\jupyter\runtime\kernel-68a914d7-9f65-41cb-a19b-cb07dad5c307 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\1\AppData\Roaming\jupyter\runtime\kernel-68a914d7-9f65-41cb-a19b-cb07dad5c307'

----------------------------------------------------------------------
Ran 1 test in 0.004s

FAILED (errors=1)


SystemExit: True

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
